In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "hockey4"
host = 'nv-desktop-services.apporto.com'
port = 32418
database = "AAC"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Animal Shelter Dashboard'))),
    html.Center(html.B(html.H2('Chad Nadeau'))),
    html.A(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': 350,
                    'width': 350,
                    'display': 'block',
                    'margin-left': 'auto',
                    'margin-right': 'auto'
            },
            alt="Loading..."
        ),
            href='https://snhu.edu'
    ),
    html.Hr(),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water', 'value': 'Water'},
            {'label': 'Mountain', 'value': 'Mountain'},
            {'label': 'Disaster', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value=None,
        labelStyle={'display': 'inline-block'}
    ),
    html.Br(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ]
             ),
    html.Hr(),
])
#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id', 'data'),
              Output('datatable-id', 'columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type is None:
        return df.to_dict('records'), [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    if filter_type == 'Reset':
        filter_query = {}
    elif filter_type == 'Water':
        filter_query = {
            'breed': {'$in': [
                'Labrador Retriever Mix', 
                'Chesa Bay Retr Mix',
                'Newfoundland Mix',
                'Newfoundland/Labrador Retriever',
                'Newfoundland/Australian Cattle Dog',
                'Newfoundland/Great Pyrenees'
            ]},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 25, '$lte': 157}
        }
    elif filter_type == 'Mountain':
        filter_query = {
            'breed': {'$in': [
                'German Shepherd',
                'Alaskan Malamute',
                'Old English Sheepdog',
                'Siberian Husky',
                'Rottweiler'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 25, '$lte': 157}
        }
    elif filter_type == 'Disaster':
        filter_query = {
            'breed': {'$in': [
                'Doberman Pinsch',
                'Doberman Pinsch Mix',
                'German Shepherd',
                'Golden Retriever',
                'Golden Retreiver Mix',
                'Bloodhound',
                'Rottweiler'
            ]},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 19, '$lte': 301}
        }

    filtered_data = db.read(filter_query)
    filtered_df = pd.DataFrame.from_records(filtered_data)
    filtered_df.drop(columns=['_id'], inplace=True)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in filtered_df.columns]
    data = filtered_df.to_dict('records')

    return data, columns

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data"),
     Input('filter-type', 'value')])
def update_graphs(data, filter_type):
    if filter_type is None or filter_type == 'Reset':
        return []
        
    dff = pd.DataFrame.from_records(data)
    
    if not dff.empty:
        names = dff['breed'].value_counts().keys().tolist()
        values = dff['breed'].value_counts().tolist()
        return [
            dcc.Graph(            
                figure = px.pie(
                names = names, 
                values = values, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
                )
            )
        ]
    else:
        return []
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



print('Connection Successful!')
app.run_server(debug=True)

Connection Successful!
Dash app running on http://127.0.0.1:26774/
